<a href="https://colab.research.google.com/github/diptivs/machine_learning/blob/master/NELA_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SJSU ML 2019 : CMPE 257

Professor: Dr. Arsanjani

Team: Cloud_stocks

Members: 
* Dipti Shiralkar
* Parvizsho Aminov
* Meghana Yoganarasimha
* Navjot Bola 

#Details of dataset and notes:



**Datasets used:**
* We used NELA datasets available here: https://dataverse.harvard.edu/dataverse/nela
* Used 2018 dataset for rest of the features
* Used 2017 dataset to prepare a model for "Posts/Social Activities". *[ As Scraper code is changed. 2017 data file has full HTML with all tags which made social media (Facebook) data was available. But 2018 data files just have the articles]*





**Refernces:**

* https://arxiv.org/pdf/1803.10124.pdf  --> to understand the feature extraction done on 2017 dataset
* https://arxiv.org/pdf/1904.01546.pdf --> to understand the Labels.csv better


#Imports

In [0]:
import sqlite3
import string
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from matplotlib import pyplot as plt

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')
import nltk
nltk.download('punkt')

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Load the dataset DB file

In [0]:
# Setup google auth
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
for fname,fid in {"articles.db":"1K6XNG1N67fJhv65LshmT8PZIUPk61o_0"}.items():
  downloaded = drive.CreateFile({'id':fid}) 
  downloaded.GetContentFile(fname)
  articles_db =  'articles.db'
  con = sqlite3.connect(articles_db)

#Data preparation

Amalgamated + filtered final dataset is available here: nela_score_values_reduced_ds.csv : https://drive.google.com/open?id=1-MdGo8vMMLWMZBcb_K0dKz56FzNEra7W

### Exclude Sources for which Labels are not available

List of Unlabelled resources: https://drive.google.com/open?id=1ZqH4iHU8DQiNTEvZuQB7RYy5fVZLqAW7

In [0]:
def execute_sql_cmd(statement):
  filtered_data = pd.read_sql_query(statement,con)
  return filtered_data

In [0]:
excluded_sources = ('Anonymous Conservative', 'BBC UK', 'Channel 4 UK', 'Common Dreams', 'Conservative Tribune', 'Crikey', 'Delaware Liberal', 'Dick Morris Blog', 'Fort Russ', 'Freedom-Bunker', 'Hit and Run', 'Hullabaloo Blog', 'Informnapalm', 'JewWorldOrder', 'LabourList', 'Liberal Democrat Voice' ,'Losercom', 'Mail', 'Mint Press News', 'Newsnet Scotland', 'Newswars', 'OSCE', 'Politicalite', 'Politicscouk', 'Prepare For Change', 'Slugger OToole', 'The Daily Blog', 'The Daily Echo', 'The Gaurdian UK', 'The Huffington Post UK', 'The Inquisitr', 'The Manchested Evening News', 'The Week UK', 'Trump Times', 'Unian', 'Window on Eurasia Blog', 'Wizbang', 'rferl', 'theRussophileorg')

In [0]:
filtered_data = execute_sql_cmd("select * from articles where source not in ('Anonymous Conservative', 'BBC UK', 'Channel 4 UK', 'Common Dreams', 'Conservative Tribune', 'Crikey', 'Delaware Liberal', 'Dick Morris Blog', 'Fort Russ', 'Freedom-Bunker', 'Hit and Run', 'Hullabaloo Blog', 'Informnapalm', 'JewWorldOrder', 'LabourList', 'Liberal Democrat Voice' ,'Losercom', 'Mail', 'Mint Press News', 'Newsnet Scotland', 'Newswars', 'OSCE', 'Politicalite', 'Politicscouk', 'Prepare For Change', 'Slugger OToole', 'The Daily Blog', 'The Daily Echo', 'The Gaurdian UK', 'The Huffington Post UK', 'The Inquisitr', 'The Manchested Evening News', 'The Week UK', 'Trump Times', 'Unian', 'Window on Eurasia Blog', 'Wizbang', 'rferl', 'theRussophileorg')")

In [0]:
filtered_data.head()

,date,source,name,content
0,2018-02-01,Addicting Info,Donald Trump Jr Likes Fox News Tweet About Spr...,Since Donald Trump Jr. has admitted on Twitter...
1,2018-02-01,Addicting Info,REPORT UK Will Share Less Confidential Info Wi...,The Republican-written and Republican-altered ...
2,2018-02-01,Addicting Info,Trump Jr Mocked To Oblivion After Democracy Di...,The world is still laughing at the Trump admin...
3,2018-02-01,BBC,Battling to save the worlds bananas,Visiting the Matanuska banana plantation is no...
4,2018-02-01,BBC,Escape from the asylum,"In Croatia, thousands of people with mental il..."


In [0]:
print(filtered_data.shape)
print(filtered_data.size)
unique_sources = filtered_data.source.unique()
for i in unique_sources:
  print(i)

(622662, 4)
2490648
Addicting Info
BBC
Business Insider
DC Gazette
Daily Kos
Daily Signal
FiveThirtyEight
Freedom Outpost
FrontPage Magazine
Fusion
Glossy News
Humor Times
Infowars
Intellectual Conservative
Intellihub
Investors Business Daily
Observer
Politico
RT
Slate
The American Conservative
The Atlantic
The Chaser
The Guardian
The Political Insider
The Shovel
True Pundit
Veterans Today
The Washington Examiner
Yahoo News
Counter Current News
Real News Right Now
Salon
The Borowitz Report
The Spoof
Waking Times
Activist Post
CBS News
Media Matters for America
Palmer Report
Politicus USA
The Beaverton
Faking News
News Busters
Breitbart
CNBC
CNN
Shadow Proof
The Conservative Tree House
The Gateway Pundit
The Huffington Post
The Michelle Malkin Blog
USA Today
Bipartisan Report
CNS News
Daily Mail
Drudge Report
NPR
National Review
Natural News
New York Daily News
New York Post
News Biscuit
RedState
Shareblue
Talking Points Memo
Daily Beast
The Daily Caller
The New York Times
The Right Sco

##Amalagamation
Amalgamated dataset is available here: nela_aggregate_labels.csv (https://drive.google.com/open?id=1xHUgscjHWsPAuDocVqCF1ckfsI6KTt2k)

### Load labels CSV with aggregated values

Below is the approach followed for aggregation all labels in Label.csv:
* Converted each score into the range of 1(bad) to 5(good)
  * NewsGuard: ROUND(NewsGuard_Score/20))
  * Pew Research: -1→ 1, 0 → 3, 1-->5
  * Wikipedia 1-->1
  * Open Sources : purple → 1, orange/gray → 3, green → 5
  * Media Bias / Fact Check, factual_reporting → already in 1-5 range
  * Politifact → consider all false as negative numbers and true as positives→ Round(((Sum_of_values+6)/8)*5)
* **Aggregated score** = Sum_of_all_available_scores_for_source/number_of_scores_available

All the calculations are available here: https://docs.google.com/spreadsheets/d/1yn0KaqP5b0WUJYNohy3fFXHxZiRRPEWcXiDbahvtETU/edit?usp=sharing 

In [0]:
#aggregated_labels.csv: https://drive.google.com/open?id=1e_H3EaaSoupdnMTL132s6YyrwDvSR_Ib
labels_filename = 'aggregated_labels.csv'
for fname,fid in {"aggregated_labels.csv": "1e_H3EaaSoupdnMTL132s6YyrwDvSR_Ib"}.items():
  downloaded = drive.CreateFile({'id':fid}) 
  downloaded.GetContentFile(fname)
  aggregated_labels = pd.read_csv(labels_filename)

In [0]:
aggregated_labels.head()

,Source,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score",(1-5) Score,"NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total",(1-5) Score.1,"Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake",(1-5) Score.2,"Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Open Sources, aggregated fake",(1-5) Score.3,"Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True",PF (1-5) Score,aggregated,Unnamed: 65
0,The Beaverton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,orange,3.0,satire,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,-2.0,0.0,0.0,0.0,0.0,2.0,2.500000,NaN
1,Infowars,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,25.0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,purple,1.0,conspiracy_pseudoscience,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Right,3579.0,782.0,absolutely agree,right,-2.0,-1.0,0.0,1.0,0.0,0.0,2.0,1.200000,NaN
2,Breitbart,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,57.0,3,0.0,0.0,0.0,3.0,-1.0,-1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,purple,1.0,questionable_source,NaN,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,Right,11911.0,6546.0,agree,right,-2.0,-2.0,0.0,0.0,1.0,0.0,2.0,2.250000,NaN
3,Drudge Report,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,29.5,1,0.0,0.0,0.0,3.0,-1.0,-1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,purple,1.0,right_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Right,1631.0,994.0,agree,right,-2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.800000,NaN
4,Bipartisan Report,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,17.5,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,purple,1.0,questionable_source,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,0.0,-1.0,0.0,0.0,0.0,0.0,2.0,1.333333,NaN


### Join Labels CSV with articles DB using "Source" as key

In [0]:
import sqlite3

#Make the db in memory
conn = sqlite3.connect(':memory:')

filtered_data.to_sql('data', conn, index=False)
aggregated_labels.to_sql('label', conn, index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [0]:
query = '''
SELECT D.*, L.aggregated
FROM data D JOIN label L ON D.source = L.Source
'''

aggr_df = pd.read_sql_query(query, conn)
conn.close()



In [0]:
aggr_df.to_csv("nela_aggregate_labels.csv", header=True)

In [0]:

from google.colab import drive
drive.mount('drive')

!cp nela_aggregate_labels.csv drive/My\ Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


##Data Filteration

The NELA dataset is very big in size ~700K. The next challenge was to reduce it to ~20k with even distribution of labels and all available sources. Below is the approach followed for it:

- There are total 5 labels : 1 (Pants-on-fire) to 5 (True)
- Count number of sources for each label
- No. of articles per label = total dataset size(20k) / no_of_labels(5) = 4000
- Articles per source = 4000/no_of_articles_for_label

The calculation is shown below and programmed using SQL queries:


| Label | 1 | 2 | 3 | 4 | 5|
| --- | ---| ---| ---| ---| ---|
| No_of_sources | 14| 31| 33| 45| 27|
| articles_per_source | 4000/14=286 | 4000/31=129| 4000/33=121| 4000/45=89| 4000/27=148|


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_id = '1vqknNKpwpe8OsvPMaTJvMNsczdV4owHk'
downloaded = drive.CreateFile({'id':file_id}) 
downloaded.GetContentFile('nela_aggregate_labels.csv') 
data_in = pd.read_csv('nela_aggregate_labels.csv')
data_in.head()

In [0]:
conn = sqlite3.connect(':memory:')
aggr_df.to_sql('aggregate', conn, index=False)

###SQL query to filter data as per equation

`data_filtering` function is used for filtering data for each label (1-5) and evenly selecting data for each source present for that label. The function constructs a `UNION` of multiple queries that get detail for each source.

In [0]:
def data_filtering(sources, label, limit):
  query = '''SELECT * FROM ( SELECT * FROM aggregate X
  WHERE ROUND(X.aggregated) = {arg} AND source = "{src}" LIMIT 0, {lmt})
  '''
  query_parts = []
  for source in sources: 
    query_parts.append(query.format(arg=label, src=source, lmt=limit))

  query = " UNION ".join(query_parts)
  df = pd.read_sql_query(query, conn)
  return df

### Filter articles having label as 1

In [0]:
# LABEL 1
sources = ["DC Gazette",
            "Daily Stormer",
            "Faking News",
            "Freedom Outpost",
            "FrontPage Magazine",
            "LewRockwell",
            "Real News Right Now",
            "The D.C. Clothesline",
            "The Duran",
            "True Activist",
            "Pamela Geller Report",
            "The Conservative Tree House",
            "Infowars",
            "Bipartisan Report" ]
            
df = data_filtering(sources, 1, 300)

In [0]:
print(df.shape)
df.head()
df.to_csv("nela_score_1_values.csv", header=True)
!cp nela_score_1_values.csv drive/My\ Drive/

(3718, 5)


,date,source,name,content,aggregated
0,2018-02-01,DC Gazette,SHOCKER Key GOP House Member TREY GOWDY Not Se...,"It wasnt exactly widely known, but South Carol...",1.0
1,2018-02-01,DC Gazette,Saudi Arabia Seizes BILLIONS In Anti-Corruptio...,Amazing what happens when a government activel...,1.0
2,2018-02-01,Freedom Outpost,10 Amazing Facts About Americas Economic Recov...,If the U.S. economy continues to surge under P...,1.0
3,2018-02-01,Freedom Outpost,Democrats Have Absolutely No Class They Put It...,After observing the despicable behavior of the...,1.0
4,2018-02-01,Freedom Outpost,ReleaseTheMemo Even Democrats Are Admitting Th...,Never before has there been so much talk about...,1.0


### Filter articles having label as 2

In [0]:
# LABEL 2
sources = ["Freedom Daily",
          "Natural News",
          "Activist Post",
          "HumansAreFree",
          "Instapundit",
          "Intellihub",
          "NODISINFO",
          "Prison Planet",
          "Waking Times",
          "GlobalResearch",
          "Live Action",
          "Palmer Report",
          "Sputnik",
          "TheAntiMedia",
          "Drudge Report",
          "Daily Mail",
          "The Gateway Pundit",
          "Addicting Info",
          "CNS News",
          "Counter Current News",
          "Pravada Report",
          "RedState",
          "Russia-Insider",
          "Shareblue",
          "The Daily Express",
          "The Right Scoop",
          "Veterans Today",
          "Western Journal",
          "21stCenturyWire",
          "Breitbart",
          "True Pundit"]

df = data_filtering(sources, 2, 150)


In [0]:
print(df.shape)
df.head()
df.to_csv("nela_score_2_values.csv", header=True)
!cp nela_score_2_values.csv drive/My\ Drive/

(4168, 5)


### Filter articles having label as 3

In [0]:
# LABEL 3
sources = ["The Beaverton",
          "The Political Insider",
          "Daily Kos",
          "RT",
          "Politicus USA",
          "The Daily Caller",
          "TheBlaze",
          "Raw Story",
          "Al Jazeera",
          "AMERICAblog News",
          "Glossy News",
          "Humor Times",
          "News Biscuit",
          "ScrappleFace",
          "The Borowitz Report",
          "The Chaser",
          "The Onion",
          "The Poke",
          "The Shovel",
          "The Spoof",
          "Daily Signal",
          "FT Westminster Blog",
          "Birmingham Mail",
          "Feministing Blog",
          "News Busters",
          "RightWingWatch",
          "sott.net",
          "Telesur TV",
          "The Daily Mirror",
          "The Daily Record",
          "The Daily Star",
          "The Sun",
          "New York Post" ]

df = data_filtering(sources, 3, 130)

In [0]:
print(df.shape)
df.head()
df.to_csv("nela_score_3_values.csv", header=True)
!cp nela_score_3_values.csv drive/My\ Drive/

(3824, 5)


### Filter articles having label as 4

In [0]:
#LABEL 4
sources = [ "The Washington Examiner",
          "Buzzfeed",
          "ThinkProgress",
          "Observer",
          "Bearing Arms",
          "Crooks and Liars",
          "New York Daily News",
          "The Intercept",
          "MotherJones",
          "The Huffington Post",
          "CNN",
          "Fox News",
          "Investors Business Daily",
          "Newsweek",
          "National Review",
          "Fusion",
          "Alternet",
          "Daily Beast",
          "Slate",
          "Democracy 21",
          "Evening Standard",
          "Foreign Policy",
          "Forward Progessives",
          "France24",
          "Hot Air",
          "Interpreter Mag",
          "iPolitics",
          "Media Matters for America",
          "Pink News UK",
          "Salon",
          "Shadow Proof",
          "SkyNewsPolitics",
          "SkyNewsUS",
          "Spiegel",
          "Talking Points Memo",
          "Tass",
          "The Fiscal Times",
          "The Independent",
          "The Irish Times",
          "The Moscow Times",
          "The Telegraph",
          "Wings Over Scotland",
          "MSNBC",
          "The New York Times",
          "Yahoo News" ]

df = data_filtering(sources, 4, 100)

In [0]:
print(df.shape)
df.head()
df.to_csv("nela_score_4_values.csv", header=True)
!cp nela_score_4_values.csv drive/My\ Drive/

(4352, 5)


### Filter articles having label as 5

In [0]:
# LABEL 5
sources = ["Business Insider",
          "Chicago Sun-Times",
          "CNBC",
          "FiveThirtyEight",
          "Fortune",
          "Mercury News",
          "Real Clear Politics",
          "The American Conservative",
          "The Atlantic",
          "The Denver Post",
          "The Hill",
          "The Verge",
          "Vox",
          "Washington Monthly",
          "ABC News",
          "BBC",
          "CBS News",
          "New Yorker",
          "PBS",
          "Politico",
          "The Guardian",
          "USA Today",
          "Washington Post",
          "oann",
          "NPR",
          "Reuters",
          "WSJ Washington Wire"]
df = data_filtering(sources, 5, 150)

In [0]:
print(df.shape)
df.head()
df.to_csv("nela_score_5_values.csv", header=True)
!cp nela_score_5_values.csv drive/My\ Drive/

(3979, 5)


### Generate a single dataset with evenly distributed labels

The goal of filtering was to generate a subset of NELA dataset that has evenly distributed data for each label. After retrieving data for each label, all datasets that were generated must be merged into one. Below code performs that and creates a single `nela_score_values_reduced_ds.csv`

In [0]:
dfs = []
for i in range(1, 6):
  dfs.append(pd.read_csv("nela_score_{}_values.csv".format(i)))
master_df = pd.concat(dfs)

In [0]:
master_df.to_csv("nela_score_values_reduced_ds.csv", header=True)
!cp nela_score_values_reduced_ds.csv drive/My\ Drive/